# [5-4] motionDetect
화면에서 움직이는 물체를 감지합니다.  
이 문서는 openCV를 기반으로 프레임 내에서 움직이는 또는 변경되는 물체가 있는지를 분석하는 데 사용됩니다.  

## Import camera function libraries

다음 코드 블록을 실행한 후에는 잠시 기다렸다가 카메라가 초기화되기를 기다리십시오.  
초기화가 성공하면 300x300 크기의 실시간 비디오 화면이 코드 블록 아래에 나타납니다. 
이 화면을 마우스 오른쪽 버튼으로 클릭하고 출력용 새 보기 만들기를 클릭하여 카메라 화면을 다시 창에 배치할 수 있습니다.  
문서의 다른 부분을 찾아도 언제든지 카메라 화면을 볼 수 있습니다. 이 방법은 다른 위젯에도 적용됩니다.  
이 코드 블록을 여러 번 실행하여 초기화가 실패할 수 있습니다. 해결 방법은 이미 jetbot.Camera에 포함되어 있으며, 커널을 다시 시작해야 합니다.  
하지만 탭 위의 원 모양 화살표를 사용하지 않도록 주의하세요. 카메라가 여전히 초기화에 실패할 수 있습니다.  
커널을 다시 시작하는 권장 방법은 다음과 같습니다:   
 - 왼쪽의 파일 브라우저에서 앞에 녹색 점이 있는 *.ipynb 파일을 마우스 오른쪽 버튼으로 클릭하고 커널 종료를 선택하면 녹색 점이 사라지고, 이 탭을 닫고 방금 닫은 *.ipynb 파일을 두 번 클릭하여 커널을 다시 시작할 수 있습니다.

다음 코드를 다시 실행하면 카메라가 정상적으로 초기화됩니다.

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)
#카메라 객체와 크기를 맞출 필요는 없다.
image_widget = ipywidgets.Image()
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Motion detection function

움직임 감지 기능은 openCV를 기반으로 합니다.  
Jetpack에는 openCV가 미리 설치되어 있으므로 다음 코드 블록을 실행하여 필요한 함수 라이브러리를 직접 가져올 수 있습니다.  
Jetpack을 사용하지 않는 경우에는 터미널에서 openCV 또는 imutils를 수동으로 설치해야 할 수 있으며, 각각 sudo pip3 install opencv-python 및 sudo pip3 install imutils를 사용하여 라이브러리를 설치할 수 있습니다.  
이 두 라이브러리가 없다는 오류 메시지가 나오지 않으면 무시하고 다음 코드 블록을 진행할 수 있습니다.

In [6]:
import cv2
import imutils
import datetime

#avg 변수는 참조 이미지(배경)를 저장하는 데 사용됩니다.
#새로운 이미지는 이것과 비교되어 이미지에서 어디가 변경되었는지를 결정합니다.
avg = None

lastMovtionCaptured = datetime.datetime.now()

# Motion 감지 함수
def motionDetect(imgInput):
    global avg, lastMovtionCaptured
    
    # 현재 시간 정보 가져오기
    timestamp = datetime.datetime.now()
    
    # 분석 향상을 위해 흑백 이미지로 변환하기
    gray = cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY)
    
# 프레임에 가우시안 블러를 적용하여 잡음에 의한 오해를 피합니다.
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # 참조 프레임(배경)이 아직 얻어지지 않았다면, 새로운 것을 생성합니다.
    if avg is None:
        avg = gray.copy().astype("float")
        return imgInput

    # 배경 업데이트 하기
    cv2.accumulateWeighted(gray, avg, 0.5)
    
    # 배경과 새로운 프레임을 비교합니다.
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))

    # 프레임에서 변화된 영역의 윤곽선을 가져옵니다.
    thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # 프레임 내에는 하나 이상의 영역이 변경될 수 있으므로, 모든 윤곽선을 얻기 위해 for 루프를 사용해야 합니다.
    for c in cnts:
        #여기서의 기본값은 30으로, 변경 영역의 임계값입니다. 우리는 800보다 큰 영역만을 분석합니다.
        #값이 작을수록 움직임 감지가 민감해지지만, 무의미한 잡음도 감지할 수 있습니다.
        if cv2.contourArea(c) < 30:
            continue

        # 직사각형 및 텍스트와 같은 요소를 그립니다.
        (mov_x, mov_y, mov_w, mov_h) = cv2.boundingRect(c)
        cv2.rectangle(imgInput, (mov_x, mov_y), (mov_x+mov_w, mov_y+mov_h), (128, 255, 0), 1)

        # 변경이 감지된 시간을 표시하기 위해 현재 타임스탬프를 저장합니다.
        lastMovtionCaptured = timestamp

    # 요소를 그리는 빈도가 너무 높아지는 것을 방지하기 위해 움직임이 끝난 후 0.5초 동안은 요소가 유지됩니다.
    if (timestamp - lastMovtionCaptured).seconds >= 0.5:
        cv2.putText(imgInput,"Motion Detecting", (10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(128,255,0),1,cv2.LINE_AA)
    else:
        cv2.putText(imgInput,"Motion Detected", (10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,128,255),1,cv2.LINE_AA)
    # 처리된 프레임을 반환합니다.
    return imgInput

## Process video frames and display

다음 코드를 실행한 후에는 프레임의 색상이 변경된 것을 볼 수 있습니다.  
이는 video screen이 motionDetect() 함수에 의해 성공적으로 처리되었음을 나타냅니다.

In [7]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(motionDetect(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

이 시점에서 모든 코드를 실행했습니다.  
프레임 내에서 객체가 움직이거나 변경될 때, 텍스트 내용이 변경되고, 변경된 영역에는 녹색 사각형이 표시됩니다.

## Turn off this processing and stop the camera

다음 코드를 실행하여 이미지 처리 기능을 종료하세요.  

In [8]:
camera.unobserve(execute, names='value')

다시 한 번 카메라 연결을 적절하게 종료하여 나중에 노트북에서 카메라를 사용할 수 있도록 합시다.

In [9]:
camera.stop()